# Demo 4 — Counterfactual COA Analysis

**OODA Phase:** Orient / Decide (stress-testing the boundary)

**Purpose:** When a planner accepts or rejects a proposed course of action, this system generates "nearby worlds" — scenarios differing by a single assumption — and tests whether the preference holds under those small perturbations. If the preference is inconsistent, it identifies the likely cognitive bias by name and presents a structured "appeal brief" explaining what changed, what stayed constant, and why the recommendation does or doesn't remain valid.

**Audience:** Military planners, wargaming staff, decision-science researchers — for research and educational purposes only
**Primary outcome:** Surface hidden cognitive biases in COA selection through counterfactual reasoning and multi-agent analysis

> **Responsible AI & Scope Statement:** This research explores human-AI collaboration and explainable decision-support in fully synthetic, abstract environments. All scenarios, agents, and data are artificially generated. No real-world operational data, contingency plans, systems, or intelligence are used or represented. The prototype is intended exclusively for research, educational, and experimental purposes and does not constitute an operational model, validated planning tool, or source of decision authority.

## What It Illustrates (Multi-Agent, AutoGen 0.7.5)

| Agent | Role |
|-------|------|
| **COA Generator** | Suggests 2–3 meaningfully different courses of action (aggressive / deliberate / indirect) with rationale, risk, and resource estimates |
| **Adjudication Agent** | Scores every COA via a transparent, **published** rule-based rubric — not a black box |
| **Counterfactual Perturbation Agent** | Varies one assumption at a time (weather, readiness, adversary intent, ROE) to create "nearby worlds" |
| **Bias / Feedback Auditor** | Compares accept/reject patterns across worlds; labels cognitive biases (anchoring, recency, escalation commitment, mirror-imaging, availability heuristic) |
| **Explainability Agent** | Produces structured "appeal briefs" for each flagged inconsistency |

**Orchestration:** AutoGen 0.7.5 `AssistantAgent` instances coordinated in a phased pipeline with planner-in-the-loop interaction

**Success criteria:** The system detects at least one bias pattern in the planner's choices and produces a cogent appeal brief


## Azure Technologies Used in This Demo

This notebook relies on several Azure services working together to power a multi-agent AI system. If you are new to Azure, here is a brief overview of each technology and how it fits into the demo.

| Technology | What It Does in This Demo | Learn More |
|---|---|---|
| **Azure AI Foundry** | Hosts the large language model (GPT-4o) that each agent calls for reasoning, scoring, and analysis. Provides a managed endpoint so you don't need to run GPU hardware yourself. | [Azure AI Foundry overview](https://learn.microsoft.com/azure/ai-studio/what-is-ai-studio) |
| **Azure AI Model Inference API** | A unified REST API for sending prompts to models deployed in Azure AI Foundry. The `AzureAIChatCompletionClient` in AutoGen wraps this API. | [Azure AI Model Inference API](https://learn.microsoft.com/azure/ai-studio/reference/reference-model-inference-api) |
| **Azure Key Vault** | Securely stores API keys and connection strings so they never appear in source code. This project uses `DefaultAzureCredential` to read secrets automatically. | [Azure Key Vault overview](https://learn.microsoft.com/azure/key-vault/general/overview) |
| **Azure Machine Learning (AML)** | Provides the cloud compute environment (notebook VMs, managed compute) where this notebook can run. The bootstrap code handles AML-specific mount paths. | [Azure Machine Learning overview](https://learn.microsoft.com/azure/machine-learning/overview-what-is-azure-machine-learning) |
| **Managed Identity & DefaultAzureCredential** | Lets the notebook authenticate to Key Vault and AI Foundry *without* embedding passwords or keys in code. In Azure it uses Managed Identity; locally it falls back to `az login`. | [DefaultAzureCredential](https://learn.microsoft.com/python/api/overview/azure/identity-readme#defaultazurecredential) |
| **AutoGen 0.7.5 (Microsoft)** | An open-source multi-agent orchestration framework from Microsoft Research. The `autogen-ext[azure]` extension adds native support for Azure AI Foundry model endpoints. | [AutoGen documentation](https://microsoft.github.io/autogen/) |

> **Tip for newcomers:** You do not need deep Azure expertise to run this demo. The two key requirements are (1) an Azure AI Foundry endpoint with a deployed model, and (2) the corresponding API key — both stored securely in Azure Key Vault or set as environment variables.

## Demo Script (What You Say / Do)

1. **Intro** — "We've seen AI suggest options and support coordination across echelons in fully synthetic scenarios. Now we stress-test *the planner's own reasoning*."
2. **Show config** — Walk through the synthetic tactical situation and the five scoring dimensions (published rubric). Emphasize that all data is artificially generated.
3. **Phase 1 — COA Generation** — COA Generator proposes 3 COAs; Adjudication Agent scores them transparently. All outputs are recommendations, not directives.
4. **Phase 2 — Baseline Decisions** — *Invite an audience member* to accept or reject each COA. Record choices.
5. **Phase 3 — Counterfactual Worlds** — Perturbation Agent creates 4 "nearby worlds," each differing by one variable.
6. **Phase 4 — Re-evaluate** — Same audience member decides again in each perturbed world.
7. **Phase 5 — The Reveal** — Bias Auditor surfaces inconsistencies; Explainability Agent delivers appeal briefs.
8. **Close** — "The AI didn't replace the planner — it made the planner's *reasoning visible* for reflection and learning."

## Setup / Install

Run once per environment. Requires **AutoGen 0.7.5** (`autogen-agentchat`, `autogen-ext[openai]`) and `ipywidgets`.

**What the packages do:**
- **`autogen-agentchat`** — Microsoft's open-source framework for building multi-agent AI applications. Each "agent" in this demo is an `AssistantAgent` backed by a large language model. ([AutoGen on PyPI](https://pypi.org/project/autogen-agentchat/))
- **`autogen-ext[azure]`** — An extension that adds `AzureAIChatCompletionClient`, which connects AutoGen agents to models deployed in [Azure AI Foundry](https://learn.microsoft.com/azure/ai-studio/what-is-ai-studio) via the [Azure AI Model Inference API](https://learn.microsoft.com/azure/ai-studio/reference/reference-model-inference-api).
- **`python-dotenv`** — Loads environment variables from a local `.env` file so you can set `AZURE_INFERENCE_ENDPOINT` and `AZURE_INFERENCE_CREDENTIAL` without hard-coding secrets.
- **`ipywidgets`** — Provides interactive buttons and controls used for the planner-in-the-loop accept/reject decisions.

In [1]:
# Uncomment to install dependencies
# %pip install -U "autogen-agentchat==0.7.5" "autogen-ext[azure]==0.7.5" python-dotenv ipywidgets

In [ ]:
# ═══════════════════════════════════════════════════════════════
# NAML 2026 BOOTSTRAP v2 — Survives dead AML mounts (Errno 107)
# ═══════════════════════════════════════════════════════════════

import os
import sys

def _safe_stat(path: str) -> bool:
    try:
        os.stat(path)
        return True
    except OSError:
        return False

def _prune_dead_sys_path():
    kept = []
    removed = []
    for p in list(sys.path):
        if not p:
            kept.append(p)
            continue
        if _safe_stat(p):
            kept.append(p)
        else:
            removed.append(p)
    sys.path[:] = kept
    print(f"✓ Pruned sys.path. Removed {len(removed)} dead entries.")
    return removed

def _safe_listdir(path: str):
    try:
        return os.listdir(path)
    except OSError:
        return None

def _find_repo_root(marker_dir: str = "common", start_candidates=None, max_up: int = 6):
    """
    Find a repo root by looking for a marker directory (e.g., 'common').
    Avoids Path.exists()/stat on dead mounts by only using listdir on traversable dirs.
    """
    if start_candidates is None:
        start_candidates = []

    # Candidate starting points:
    #  - current working directory (may be dead)
    #  - directory of the notebook file if available via env (sometimes set)
    #  - user home (often stable)
    candidates = [os.getcwd()] + start_candidates + [os.path.expanduser("~")]

    checked = set()
    for base in candidates:
        cur = base
        for _ in range(max_up + 1):
            if cur in checked:
                break
            checked.add(cur)

            entries = _safe_listdir(cur)
            if entries is not None and marker_dir in entries:
                return cur  # found repo root

            parent = os.path.dirname(cur)
            if parent == cur:
                break
            cur = parent

    return None

# 1) prune dead sys.path entries
_prune_dead_sys_path()

# 2) find a safe repo root by locating the 'common/' folder
repo_root = _find_repo_root(marker_dir="common", start_candidates=[])

if repo_root:
    sys.path.insert(0, repo_root)
    print(f"✓ Repo root added: {repo_root}")
else:
    print("✗ Could not find repo root safely (mount may be disconnected).")
    print("  Fix: restart kernel/compute, or run from a local (non-/mnt) working copy.")

print("✓ Bootstrap complete.")


## Imports

The cells below import the Python libraries used throughout this demo. Key Azure-specific imports include:

| Import | Purpose | Documentation |
|---|---|---|
| `AzureAIChatCompletionClient` | AutoGen's client for calling models deployed in **Azure AI Foundry**. Wraps the [Azure AI Model Inference API](https://learn.microsoft.com/azure/ai-studio/reference/reference-model-inference-api) so each agent can send prompts and receive completions. | [autogen-ext Azure docs](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.models.azure.html) |
| `AzureKeyCredential` | A lightweight credential object from the Azure SDK that holds an API key for authenticating to Azure AI services. This is the simplest authentication method — pass your endpoint's key and it handles the `Authorization` header automatically. | [AzureKeyCredential reference](https://learn.microsoft.com/python/api/azure-core/azure.core.credentials.azurekeycredential) |
| `ModelFamily` | An AutoGen enum that describes the model's capabilities (vision, function calling, JSON output). Used when constructing the model client so AutoGen knows what features the deployed model supports. | [AutoGen model info](https://microsoft.github.io/autogen/stable/reference/python/autogen_core.models.html) |
| `AssistantAgent` | AutoGen's core agent class. Each agent wraps a system prompt + model client. Messages are sent via `on_messages()`, and the agent returns structured responses. | [AssistantAgent docs](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html) |

> **New to Azure authentication?** This demo uses an **API key** (via `AzureKeyCredential`). For production workloads, Microsoft recommends [Managed Identity](https://learn.microsoft.com/entra/identity/managed-identities-azure-resources/overview) with `DefaultAzureCredential` instead — no keys to manage or rotate. The shared `common/config.py` module supports both approaches.

In [ ]:
# ═══════════════════════════════════════════════════════════════
# NAML 2026 BOOTSTRAP — Safe Import Setup for Azure ML
# ═══════════════════════════════════════════════════════════════

import os
import sys
from pathlib import Path

def _prune_dead_sys_path():
    """Remove broken mount paths (common in Azure ML after remount issues)."""
    cleaned = []
    for p in sys.path:
        try:
            if not p or os.path.exists(p):
                cleaned.append(p)
        except OSError:
            # Dead mount or broken transport endpoint
            pass
    sys.path[:] = cleaned

def _add_repo_root(levels_up: int = 2):
    """
    Add repo root relative to notebook location.
    Adjust `levels_up` if folder depth changes.
    """
    repo_root = Path(os.getcwd()).resolve()
    for _ in range(levels_up):
        repo_root = repo_root.parent

    if repo_root.exists():
        sys.path.insert(0, str(repo_root))
        print(f"✓ Repo root added: {repo_root}")
    else:
        print("⚠ Repo root not found:", repo_root)

# Execute bootstrap steps
_prune_dead_sys_path()
_add_repo_root(levels_up=2)

print("✓ Bootstrap complete.")


In [2]:
import json
import os
import re
import sys
from typing import Any, Dict, List, Tuple

from IPython.display import display, HTML, Markdown, clear_output
import ipywidgets as widgets

# AutoGen 0.7.5
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.azure import AzureAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_core.models import ModelFamily
from azure.core.credentials import AzureKeyCredential

# Ensure the repo root is on the path so `common` is importable
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

# ── Common utilities ──────────────────────────────────────────
from common.config import (
    DemoID, DEMOS,
    ENV_AZURE_INFERENCE_ENDPOINT, ENV_AZURE_INFERENCE_CREDENTIAL,
    DEFAULT_MODEL,
    NUM_COUNTERFACTUAL_WORLDS as DEFAULT_NUM_COUNTERFACTUAL_WORLDS,
 )
from common.logging import (
    log_info, log_error, log_success,
    log_section, log_step, log_metric, clear_logs,
 )
from common.ui import (
    make_decision_widgets, CONFIDENCE_COLORS, render_info_box,
 )

# Optional: load .env for API keys
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass

# ── JSON parsing helper (handles markdown fences from LLMs) ──
def parse_json_response(text: str) -> Any:
    """Parse JSON from an LLM response, stripping markdown fences if present."""
    cleaned = text.strip()
    fence_match = re.search(r"```(?:json)?\s*\n?(.*?)```", cleaned, re.DOTALL)
    if fence_match:
        cleaned = fence_match.group(1).strip()
    return json.loads(cleaned)

# Load demo-specific config from the registry
DEMO_CFG = DEMOS[DemoID.COUNTERFACTUAL_COA]
log_success(f"Imports ready — {DEMO_CFG.title}")

LogEntry(level='SUCCESS', message='Imports ready — Counterfactual COA Analysis', timestamp='2026-02-12 19:15:08', extra={})

## LLM Configuration

Configure your Azure AI Foundry model endpoint below. AutoGen 0.7 uses explicit `model_client` objects rather than global config.

### What is Azure AI Foundry?

[Azure AI Foundry](https://learn.microsoft.com/azure/ai-studio/what-is-ai-studio) (formerly Azure AI Studio) is Microsoft's unified platform for building generative AI applications. Think of it as a "control plane" where you can:

1. **Deploy models** — Select from a catalog of foundation models (OpenAI GPT-4o, Mistral, Llama, etc.) and deploy them to a managed endpoint with a single click.
2. **Get an endpoint + key** — Each deployment gives you a REST URL (`AZURE_INFERENCE_ENDPOINT`) and an API key (`AZURE_INFERENCE_CREDENTIAL`). These are the two values this notebook needs.
3. **Monitor usage** — Track token consumption, latency, and errors through built-in dashboards.

### How the code connects to Azure

```
┌──────────────┐     TextMessage      ┌──────────────────────────────┐     HTTPS / REST     ┌──────────────────────┐
│  AutoGen     │ ──────────────────►  │  AzureAIChatCompletionClient │ ──────────────────►  │  Azure AI Foundry    │
│  Agent       │                      │  (wraps Inference API)       │                      │  GPT-4o endpoint     │
└──────────────┘  ◄──────────────────  └──────────────────────────────┘  ◄──────────────────  └──────────────────────┘
                   JSON response                                          completion
```

### Required environment variables

| Variable | Where to find it | Docs |
|---|---|---|
| `AZURE_INFERENCE_ENDPOINT` | Azure AI Foundry → your project → Deployments → select model → copy **Target URI** | [Deploy models](https://learn.microsoft.com/azure/ai-studio/how-to/deploy-models-openai) |
| `AZURE_INFERENCE_CREDENTIAL` | Same page → **Keys** section → copy **Key 1** | [Manage keys](https://learn.microsoft.com/azure/ai-studio/how-to/connections-add) |

You can set these in a `.env` file at the repo root (excluded from Git via `.gitignore`) or store them in [Azure Key Vault](https://learn.microsoft.com/azure/key-vault/general/overview) for production use.

In [3]:
# ── LLM Configuration ──────────────────────────────────────────
# Azure AI Foundry / Azure AI Inference → set AZURE_INFERENCE_ENDPOINT + AZURE_INFERENCE_CREDENTIAL

# Hard-code the model ID you want to use for this demo.
FOUNDRY_MODEL = DEFAULT_MODEL

def build_model_client():
    """Build an AutoGen 0.7 model client for Azure AI Foundry models.

    Uses constants from common.config for env-var names and defaults.
    """
    missing = [
        name for name in (ENV_AZURE_INFERENCE_ENDPOINT, ENV_AZURE_INFERENCE_CREDENTIAL)
        if not os.environ.get(name)
    ]
    if missing:
        log_error("Missing Foundry configuration: " + ", ".join(missing))
        raise EnvironmentError(
            "Missing Azure AI Foundry / Inference configuration. Set:\n"
            f"  {ENV_AZURE_INFERENCE_ENDPOINT}\n"
            f"  {ENV_AZURE_INFERENCE_CREDENTIAL}\n"
        )

    model_info = {
        "family": ModelFamily.UNKNOWN,
        "vision": False,
        "function_calling": False,
        "json_output": True,
        "structured_output": False,
        "multiple_system_messages": True,
    }

    model_client = AzureAIChatCompletionClient(
        endpoint=os.environ[ENV_AZURE_INFERENCE_ENDPOINT],
        credential=AzureKeyCredential(os.environ[ENV_AZURE_INFERENCE_CREDENTIAL]),
        model=FOUNDRY_MODEL,
        model_info=model_info,
        temperature=DEMO_CFG.temperature,
    )
    return model_client, "Azure AI Foundry", FOUNDRY_MODEL

model_client, provider_name, model_name = build_model_client()

# ── Demo Configuration ────────────────────────────────────────
NUM_COUNTERFACTUAL_WORLDS = DEFAULT_NUM_COUNTERFACTUAL_WORLDS

log_success(f"LLM configured: {provider_name} / {model_name}")

LogEntry(level='SUCCESS', message='LLM configured: Azure AI Foundry / gpt-4o', timestamp='2026-02-12 19:15:08', extra={})

## Tactical Scenario & Perturbation Axes

The **baseline** situation defines the starting world. All scenario content is **fully synthetic and fictional** — no real-world operational data, units, or intelligence are represented. Each **perturbation axis** changes exactly one variable so causal attribution is unambiguous.

> **Azure context:** The scenario data below is defined as a plain Python dictionary. It gets serialized to text and sent to the GPT-4o model hosted in Azure AI Foundry as part of each agent's prompt. Azure AI Foundry handles tokenization, inference, and response generation — you simply send text and receive text. There is no special Azure service for scenario management; the structured data lives entirely in this notebook.

In [ ]:
# ── Baseline Tactical Situation ────────────────────────────────
BASELINE_SITUATION = {
    "synthetic":       True,
    "disclaimer":      "All data is artificially generated for research and educational purposes only.",
    "location":        "Eastern Ceresia — disputed river crossing near Grid TQ-4472",
    "friendly_forces": ("Reinforced Mech Bn TF: 3× Mech Inf Co, 1× Tank Co, "
                        "1× Arty Btry, Engineer Plt (bridge-laying)"),
    "enemy_forces":    ("Reinforced Motor Rifle Bn in prepared defense: est 2× MRC, "
                        "AT plt, mortar btry; possible reserve BN within 4 hr"),
    "weather":         "Overcast, vis 3 km, light rain 0600–1200, ground soft but passable",
    "terrain":         ("Open agricultural plain N of river; urban area (pop ~8 k) on S bank; "
                        "1 intact bridge, 1 damaged ford 2 km E"),
    "mission":         "Seize crossing points NLT 081800Z FEB 26 to enable follow-on brigade passage",
    "roe":             ("Standard ROE; fires on PID mil targets; CDE required for strikes "
                        "within 500 m of urban area"),
    "comms":           "Pri SATCOM / Alt HF / Contingency: messenger",
    "logistics":       "Class V @ 80 %; Class III sufficient 48 hr; MEDEVAC < 30 min",
    "time_available":  "H-hour in 6 hr; crossing secured before dawn D+1",
}

# ── Perturbation Axes (one variable changed per "nearby world") ──
PERTURBATION_AXES = {
    "weather": {
        "label": "Weather degrades → dense fog",
        "change": {
            "weather": "Dense fog, vis < 200 m, heavy rain, ground near-impassable for wheeled vehicles"
        },
    },
    "unit_readiness": {
        "label": "Tank company at 50 % strength",
        "change": {
            "friendly_forces": ("Weakened Mech Bn TF: 3× Mech Inf Co, "
                                "1× Tank Co (50 % — 2 of 4 plt), 1× Arty Btry, Eng Plt")
        },
    },
    "adversary_intent": {
        "label": "Enemy reserves arrive in 2 hr (not 4)",
        "change": {
            "enemy_forces": ("Reinforced MR Bn in prepared defense + reserve BN arriving "
                             "in 2 hr (faster than estimated)")
        },
    },
    "roe_constraint": {
        "label": "ROE tightened — no fires within 1 km of urban area",
        "change": {
            "roe": "Restricted ROE; fires on PID mil targets; NO fires within 1 km of urban area"
        },
    },
}

def build_situation_text(situation: Dict[str, str]) -> str:
    """Format a situation dict into a readable brief."""
    return "\n".join(f"• {k.replace('_', ' ').title()}: {v}" for k, v in situation.items())

display(Markdown("### Baseline Tactical Situation\n" + build_situation_text(BASELINE_SITUATION)))

### Baseline Tactical Situation
• Location: Eastern Ceresia — disputed river crossing near Grid TQ-4472
• Friendly Forces: Reinforced Mech Bn TF: 3× Mech Inf Co, 1× Tank Co, 1× Arty Btry, Engineer Plt (bridge-laying)
• Enemy Forces: Reinforced Motor Rifle Bn in prepared defense: est 2× MRC, AT plt, mortar btry; possible reserve BN within 4 hr
• Weather: Overcast, vis 3 km, light rain 0600–1200, ground soft but passable
• Terrain: Open agricultural plain N of river; urban area (pop ~8 k) on S bank; 1 intact bridge, 1 damaged ford 2 km E
• Mission: Seize crossing points NLT 081800Z FEB 26 to enable follow-on brigade passage
• Roe: Standard ROE; fires on PID mil targets; CDE required for strikes within 500 m of urban area
• Comms: Pri SATCOM / Alt HF / Contingency: messenger
• Logistics: Class V @ 80 %; Class III sufficient 48 hr; MEDEVAC < 30 min
• Time Available: H-hour in 6 hr; crossing secured before dawn D+1

## Agent Definitions (AutoGen 0.7.5)

Five `AssistantAgent` instances — each with a focused system prompt that constrains output to structured JSON for reliable downstream parsing.

### How AutoGen agents work with Azure

[AutoGen](https://microsoft.github.io/autogen/) is an open-source multi-agent framework from Microsoft Research. In version 0.7, each agent is constructed with:

- **`name`** — A unique identifier used in logs and orchestration.
- **`model_client`** — The `AzureAIChatCompletionClient` configured earlier, which routes every LLM call to your Azure AI Foundry endpoint. All five agents share the same deployed model but behave differently because of their distinct system prompts.
- **`system_message`** — A detailed prompt that defines the agent's personality, task, and output format. This is injected as the `system` role message on every API call to the Azure endpoint.

When you call `agent.on_messages(...)`, AutoGen packages the conversation into the [Chat Completions request format](https://learn.microsoft.com/azure/ai-services/openai/reference#chat-completions), sends it to Azure AI Foundry, and returns the model's response as a `TextMessage`.

> **Key insight:** The "multi-agent" pattern here is a *software architecture* choice, not multiple Azure deployments. All five agents call the **same** GPT-4o endpoint — the differentiation comes from prompt engineering, not infrastructure. This keeps costs and operational complexity low. Learn more about multi-agent design patterns in the [AutoGen documentation](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/index.html).

In [ ]:
# ── 1. COA Generator Agent ────────────────────────────────────
coa_generator = AssistantAgent(
    name="COA_Generator",
    model_client=model_client,
    system_message=(
        "You are a synthetic Course of Action (COA) generator for a fully fictional wargaming scenario. "
        "All decisions remain with the human operator. You provide analysis and options, not directives. "
        "Given a tactical situation brief, produce exactly 3 distinct COAs representing "
        "meaningfully different approaches (aggressive, deliberate, indirect).\n\n"
        "Return ONLY a JSON array with objects containing:\n"
        '- "id": "COA_A" / "COA_B" / "COA_C"\n'
        '- "name": short descriptive name (≤8 words)\n'
        '- "approach": "aggressive" | "deliberate" | "indirect"\n'
        '- "concept": 3-4 sentence concept of operations\n'
        '- "risk": "low" | "medium" | "high"\n'
        '- "resource_focus": which capability is the main effort\n'
        '- "estimated_casualties_pct": integer 0-100\n'
        '- "time_to_complete_hr": float\n\n'
        "No markdown fences. No extra text. JSON only."
    ),
)

# ── 2. Counterfactual Perturbation Agent ──────────────────────
perturbation_agent = AssistantAgent(
    name="Perturbation_Agent",
    model_client=model_client,
    system_message=(
        "You are a counterfactual reasoning specialist operating in a fully synthetic wargaming scenario. "
        "All decisions remain with the human operator. You provide analysis and options, not directives. "
        "You receive a set of COAs "
        "and a perturbed tactical situation that differs from the baseline by exactly "
        "ONE changed variable.\n\n"
        "For each COA, analyze how the single perturbation affects its viability.\n"
        "Return ONLY a JSON array with objects:\n"
        '- "coa_id": the COA identifier\n'
        '- "perturbation": name of what changed\n'
        '- "impact_summary": 2-3 sentences on how the COA is affected\n'
        '- "revised_risk": "low" | "medium" | "high"\n'
        '- "revised_casualties_pct": integer 0-100\n'
        '- "still_viable": true/false\n\n'
        "No markdown fences. No extra text. JSON only."
    ),
)

# ── 3. Bias / Feedback Auditor Agent ─────────────────────────
bias_auditor = AssistantAgent(
    name="Bias_Auditor",
    model_client=model_client,
    system_message=(
        "You are a cognitive-bias detection specialist operating in a synthetic decision-science research environment. "
        "All decisions remain with the human operator. You provide analysis and options, not directives. "
        "You receive the planner's accept/reject decisions for COAs across a baseline world "
        "and several counterfactual worlds that each differ by ONE variable.\n\n"
        "Analyze the decision pattern. When the planner's preference flips between "
        "near-identical worlds, classify the likely bias:\n"
        "- Anchoring: over-reliance on first information\n"
        "- Recency bias: favoring the last successful approach\n"
        "- Escalation commitment: persisting despite contrary evidence\n"
        "- Mirror-imaging: assuming adversary thinks like the planner\n"
        "- Availability heuristic: overweighting vivid/recent scenarios\n\n"
        "Return ONLY a JSON array with objects:\n"
        '- "coa_id": the COA whose handling is inconsistent\n'
        '- "baseline_decision": "accept" / "reject"\n'
        '- "flipped_in_world": name of the perturbed world\n'
        '- "perturbed_decision": "accept" / "reject"\n'
        '- "variable_that_changed": what single variable differed\n'
        '- "diagnosed_bias": one of the five bias names above\n'
        '- "confidence": "low" | "medium" | "high"\n'
        '- "reasoning": 2-3 sentences explaining the diagnosis\n\n'
        "If no bias detected, return []. No markdown fences. JSON only."
    ),
)

# ── 4. Explainability Agent ───────────────────────────────────
explainability_agent = AssistantAgent(
    name="Explainability_Agent",
    model_client=model_client,
    system_message=(
        "You are a decision-support explainability specialist operating in a fully synthetic wargaming scenario. "
        "All decisions remain with the human operator. You provide analysis and options, not directives. "
        "For each bias diagnosis from the Bias Auditor, produce a structured "
        '"appeal brief".\n\n'
        "Return ONLY a JSON array with objects:\n"
        '- "coa_id": identifier\n'
        '- "brief_title": concise title for the appeal\n'
        '- "assumption_changed": what single variable was perturbed\n'
        '- "what_stayed_constant": list of variables that did NOT change\n'
        '- "outcome_shift": how the COA expected outcome changed (or didn\'t)\n'
        '- "original_preference_valid": true/false with reasoning\n'
        '- "recommendation": "reconsider" | "original stands"\n'
        '- "narrative": 3-5 sentence human-readable explanation\n\n'
        "No markdown fences. No extra text. JSON only."
    ),
)

# ── 5. Adjudication Agent ────────────────────────────────────
adjudication_agent = AssistantAgent(
    name="Adjudication_Agent",
    model_client=model_client,
    system_message=(
        "You are a COA adjudication agent for a fully synthetic wargaming scenario. "
        "All decisions remain with the human operator. You provide analysis and options, not directives. "
        "For each COA and a given "
        "tactical situation, produce scores on five dimensions (0.0 to 1.0):\n"
        "- mission_accomplishment: probability of seizing objectives on time\n"
        "- force_protection: inverse of expected friendly casualties (1.0 = minimal)\n"
        "- tempo: speed of execution relative to timeline\n"
        "- flexibility: ability to adapt if assumptions change\n"
        "- logistics_sustainability: resource consumption vs availability\n\n"
        "Return ONLY a JSON array with objects:\n"
        '- "coa_id": identifier\n'
        '- "scores": {five dimension keys with float values}\n\n'
        "No markdown fences. No extra text. JSON only."
    ),
)

agents_list = ["COA_Generator", "Adjudication_Agent", "Perturbation_Agent",
               "Bias_Auditor", "Explainability_Agent"]
log_success(f"All {len(agents_list)} AutoGen agents initialized.")
display(HTML(
    "<b>Agent Pipeline:</b> " +
    " → ".join(f"<code>{a}</code>" for a in agents_list)
))

## Adjudication Scoring Rubric (Published & Transparent)

The scoring formula is a simple weighted sum — published here so the audience can verify the adjudication is **not** a black box.

In [6]:
SCORING_RUBRIC = {
    "mission_accomplishment":  {"weight": 0.30, "desc": "P(objectives seized on time)"},
    "force_protection":        {"weight": 0.25, "desc": "Inverse of expected friendly losses"},
    "tempo":                   {"weight": 0.15, "desc": "Speed vs. timeline"},
    "flexibility":             {"weight": 0.15, "desc": "Adaptability if assumptions shift"},
    "logistics_sustainability": {"weight": 0.15, "desc": "Resource use vs. supply"},
}

def compute_weighted_score(dimension_scores: Dict[str, float]) -> float:
    """Transparent weighted-sum — anyone can verify."""
    return round(
        sum(
            SCORING_RUBRIC[d]["weight"] * dimension_scores.get(d, 0.5)
            for d in SCORING_RUBRIC
        ),
        3,
    )

# Display the rubric as an HTML table
rubric_html = (
    "<table style='border-collapse:collapse; font-size:13px;'>"
    "<tr style='background:#1f6feb;color:#fff;'>"
    "<th style='padding:6px 12px;'>Dimension</th>"
    "<th style='padding:6px 12px;'>Weight</th>"
    "<th style='padding:6px 12px;'>Description</th></tr>"
)
for dim, meta in SCORING_RUBRIC.items():
    rubric_html += (
        f"<tr><td style='padding:4px 12px;border:1px solid #ddd;'><code>{dim}</code></td>"
        f"<td style='padding:4px 12px;border:1px solid #ddd;text-align:center;'>{meta['weight']:.0%}</td>"
        f"<td style='padding:4px 12px;border:1px solid #ddd;'>{meta['desc']}</td></tr>"
    )
rubric_html += "</table>"
display(HTML("<h4>Published Adjudication Rubric</h4>" + rubric_html))

Dimension,Weight,Description
mission_accomplishment,30%,P(objectives seized on time)
force_protection,25%,Inverse of expected friendly losses
tempo,15%,Speed vs. timeline
flexibility,15%,Adaptability if assumptions shift
logistics_sustainability,15%,Resource use vs. supply


## Phase 1 — COA Generation & Scoring

The **COA Generator** suggests 3 courses of action; the **Adjudication Agent** scores each against the published rubric.

**What happens under the hood with Azure:**
1. The scenario text is packaged into a [Chat Completions](https://learn.microsoft.com/azure/ai-services/openai/reference#chat-completions) request and sent to the Azure AI Foundry endpoint.
2. The COA Generator agent's system prompt instructs the model to return structured JSON — this is a prompt-engineering technique, not a special Azure feature.
3. A second API call sends the generated COAs to the Adjudication Agent for scoring.
4. Each call costs tokens (input + output). You can monitor token usage in [Azure AI Foundry's metrics dashboard](https://learn.microsoft.com/azure/ai-studio/how-to/costs-plan-manage).

In [7]:
async def generate_and_score_coas(situation: Dict[str, str]) -> List[Dict]:
    """Phase 1: COA Generator → Adjudication Agent."""
    sit_text = build_situation_text(situation)

    # Step 1 — generate COAs
    log_step("COA_Generator", "producing 3 courses of action …")
    gen_resp = await coa_generator.on_messages(
        [TextMessage(content=f"Tactical Situation:\n{sit_text}", source="user")],
        cancellation_token=CancellationToken(),
    )
    coas = parse_json_response(gen_resp.chat_message.content)

    # Step 2 — score COAs
    log_step("Adjudication_Agent", "scoring COAs against rubric …")
    adj_prompt = f"Tactical Situation:\n{sit_text}\n\nCOAs:\n{json.dumps(coas, indent=2)}"
    adj_resp = await adjudication_agent.on_messages(
        [TextMessage(content=adj_prompt, source="user")],
        cancellation_token=CancellationToken(),
    )
    scores_list = parse_json_response(adj_resp.chat_message.content)
    score_map = {s["coa_id"]: s["scores"] for s in scores_list}

    for coa in coas:
        dim_scores = score_map.get(coa["id"], {})
        coa["dimension_scores"] = dim_scores
        coa["weighted_score"] = compute_weighted_score(dim_scores)

    return coas


# ── Run Phase 1 ──────────────────────────────────────────────
log_section("COA Generation & Scoring", phase=1,
            description="COA Generator proposes 3 COAs; Adjudication Agent scores them.")
baseline_coas = await generate_and_score_coas(BASELINE_SITUATION)

# Rich card display
for coa in baseline_coas:
    color = {"aggressive": "#d73a49", "deliberate": "#1f6feb", "indirect": "#2ea44f"}.get(
        coa.get("approach", ""), "#6e7781"
    )
    display(HTML(
        f"<div style='border:2px solid {color}; border-radius:8px; padding:12px; margin:8px 0;'>"
        f"<h4 style='color:{color}; margin:0;'>{coa['id']}: {coa['name']}</h4>"
        f"<p><b>Approach:</b> {coa['approach']} &nbsp;|&nbsp; <b>Risk:</b> {coa['risk']} &nbsp;|&nbsp; "
        f"<b>Est. casualties:</b> {coa.get('estimated_casualties_pct','?')}% &nbsp;|&nbsp; "
        f"<b>Time:</b> {coa.get('time_to_complete_hr','?')} hr</p>"
        f"<p>{coa['concept']}</p>"
        f"<p><b>Weighted Score:</b> "
        f"<span style='font-size:18px; font-weight:700;'>{coa['weighted_score']}</span></p>"
        f"</div>"
    ))

log_success(f"Phase 1 complete — {len(baseline_coas)} COAs generated and scored.")

LogEntry(level='SUCCESS', message='Phase 1 complete — 3 COAs generated and scored.', timestamp='2026-02-12 19:15:18', extra={})

## Phase 2 — Planner-in-the-Loop: Baseline Decisions

Accept or reject each COA based on the baseline situation. This is the strongest audience-engagement moment — **invite an audience member to make the choices**, then watch the system surface their reasoning patterns later.

In [8]:
baseline_decisions: Dict[str, str] = {}
phase2_output = widgets.Output()

log_section("Planner-in-the-Loop — Baseline", phase=2,
            description="Accept or reject each COA, then click Confirm.")

baseline_rows, baseline_confirm = make_decision_widgets(
    baseline_coas, "Baseline", baseline_decisions
)

def _on_baseline_confirm(_):
    with phase2_output:
        clear_output()
        for cid, dec in baseline_decisions.items():
            icon = "✅" if dec == "accept" else "❌"
            print(f"  {icon} {cid}: {dec}")
        print("\n  Baseline decisions locked.")
    log_info(f"Baseline decisions recorded: {json.dumps(baseline_decisions)}")

baseline_confirm.on_click(_on_baseline_confirm)
display(widgets.VBox(baseline_rows + [baseline_confirm, phase2_output]))

## Phase 3 — Counterfactual Perturbation

The **Perturbation Agent** creates 4 "nearby worlds," each differing from the baseline by exactly **one** variable. The **Adjudication Agent** re-scores every COA in each perturbed world so causal attribution is unambiguous.

**Azure cost note:** This phase makes **8 API calls** to Azure AI Foundry (one Perturbation + one Adjudication call per world × 4 worlds). Each call is an independent [Chat Completions](https://learn.microsoft.com/azure/ai-services/openai/how-to/chatgpt) request. Azure charges are based on the total number of [tokens](https://learn.microsoft.com/azure/ai-services/openai/concepts/tokens) processed. You can view real-time usage in the [Azure portal](https://portal.azure.com) under your AI Foundry resource's **Metrics** blade.

In [9]:
async def generate_counterfactual_worlds(
    coas: List[Dict], baseline: Dict[str, str]
) -> Dict[str, Dict]:
    """Phase 3: Perturbation Agent + Adjudication Agent per world."""
    worlds: Dict[str, Dict] = {}

    for axis_name, axis_def in PERTURBATION_AXES.items():
        perturbed_sit = {**baseline, **axis_def["change"]}
        perturbed_text = build_situation_text(perturbed_sit)

        # Perturbation assessment
        log_step("Perturbation_Agent", f"world '{axis_def['label']}' …")
        prompt = (
            f"BASELINE vs PERTURBED WORLD — only '{axis_name}' changed.\n\n"
            f"Perturbed Situation:\n{perturbed_text}\n\n"
            f"COAs to re-assess:\n{json.dumps(coas, indent=2)}"
        )
        resp = await perturbation_agent.on_messages(
            [TextMessage(content=prompt, source="user")],
            cancellation_token=CancellationToken(),
        )
        assessments = parse_json_response(resp.chat_message.content)

        # Re-score in perturbed world
        log_step("Adjudication_Agent", f"scoring in '{axis_def['label']}' …")
        adj_prompt = (
            f"Tactical Situation:\n{perturbed_text}\n\n"
            f"COAs:\n{json.dumps(coas, indent=2)}"
        )
        adj_resp = await adjudication_agent.on_messages(
            [TextMessage(content=adj_prompt, source="user")],
            cancellation_token=CancellationToken(),
        )
        perturbed_scores = parse_json_response(adj_resp.chat_message.content)
        score_map = {s["coa_id"]: s["scores"] for s in perturbed_scores}

        worlds[axis_name] = {
            "label": axis_def["label"],
            "situation": perturbed_sit,
            "assessments": assessments,
            "scores": {cid: compute_weighted_score(sc) for cid, sc in score_map.items()},
        }

    return worlds


# ── Run Phase 3 ──────────────────────────────────────────────
log_section("Counterfactual Perturbation", phase=3,
            description="One variable changed per 'nearby world'; Adjudication re-scores each.")
counterfactual_worlds = await generate_counterfactual_worlds(baseline_coas, BASELINE_SITUATION)

for axis, world in counterfactual_worlds.items():
    viability = {a["coa_id"]: a["still_viable"] for a in world["assessments"]}
    v_str = " &nbsp;|&nbsp; ".join(
        f"{c}: {'✓ viable' if v else '✗ degraded'}" for c, v in viability.items()
    )
    render_info_box(
        f"<b>World:</b> {world['label']}<br/>"
        f"<b>Viability:</b> {v_str}<br/>"
        f"<b>Weighted scores:</b> "
        + " &nbsp;|&nbsp; ".join(f"{c}: {s}" for c, s in world["scores"].items()),
        border_color="#bf8700",
    )

log_success(f"Phase 3 complete — {len(counterfactual_worlds)} counterfactual worlds generated.")

LogEntry(level='SUCCESS', message='Phase 3 complete — 4 counterfactual worlds generated.', timestamp='2026-02-12 19:15:46', extra={})

## Phase 4 — Re-evaluate COAs in Perturbed Worlds

The planner now sees the **same COAs** but in each perturbed world. Accept or reject again. Any inconsistent flips between the baseline and a perturbed world will be flagged in Phase 5.

In [10]:
perturbed_decisions: Dict[str, Dict[str, str]] = {}
phase4_output = widgets.Output()

all_world_widgets = []
world_confirms: List[Tuple[str, widgets.Button]] = []

for axis_name, world in counterfactual_worlds.items():
    perturbed_decisions[axis_name] = {}

    header = widgets.HTML(
        f"<h4 style='color:#bf8700; margin-bottom:4px;'>World: {world['label']}</h4>"
        f"<p style='font-size:12px; color:#57606a;'>Scores in this world: "
        + " | ".join(f"{c}: {s}" for c, s in world["scores"].items())
        + "</p>"
    )
    rows, confirm = make_decision_widgets(
        baseline_coas, world["label"], perturbed_decisions[axis_name]
    )
    world_confirms.append((axis_name, confirm))
    all_world_widgets.append(widgets.VBox([header] + rows + [confirm]))

confirmed_worlds: set = set()

def _make_world_confirm_handler(axis):
    def _handler(_):
        confirmed_worlds.add(axis)
        with phase4_output:
            print(f"  ✓ '{axis}' decisions locked: {perturbed_decisions[axis]}")
        if len(confirmed_worlds) == len(counterfactual_worlds):
            log_success("All perturbed-world decisions recorded.")
    return _handler

for axis_name, confirm_btn in world_confirms:
    confirm_btn.on_click(_make_world_confirm_handler(axis_name))

log_section("Re-evaluate COAs in Perturbed Worlds", phase=4,
            description="Accept or reject each COA in each perturbed world, then confirm.")
display(widgets.VBox(all_world_widgets + [phase4_output]))

## Phase 5 — Bias Detection & Explainability (The Reveal)

The **Bias Auditor** compares baseline vs. perturbed-world decisions. When it detects an inconsistency (preference flips despite a minor change), it labels the bias by name. The **Explainability Agent** then produces a detailed "appeal brief" for each finding.

> **Presenter note:** This is the strongest audience moment. The system makes the planner's *own reasoning patterns* visible.

**How Azure AI enables this analysis:** The Bias Auditor and Explainability Agent both rely on the language model's ability to reason over structured data (the JSON decision records). This is a form of **in-context learning**, where all relevant information is passed in the prompt rather than requiring fine-tuning or a separate database. Azure AI Foundry's GPT-4o model supports context windows large enough to hold the full decision matrix across all worlds. For applications that need to reason over even larger datasets, consider [Azure AI Search](https://learn.microsoft.com/azure/search/search-what-is-azure-search) to implement retrieval-augmented generation (RAG) patterns.

In [11]:
async def run_bias_analysis(
    baseline_decs: Dict[str, str],
    perturbed_decs: Dict[str, Dict[str, str]],
    coas: List[Dict],
    worlds: Dict[str, Dict],
) -> Tuple[List[Dict], List[Dict]]:
    """Phase 5: Bias Auditor → Explainability Agent."""

    # Build the full decision record
    decision_record = {"baseline": baseline_decs, "perturbed_worlds": {}}
    for axis, decs in perturbed_decs.items():
        decision_record["perturbed_worlds"][axis] = {
            "perturbation": worlds[axis]["label"],
            "decisions": decs,
        }

    # ── Bias Auditor ──────────────────────────────────────────
    log_step("Bias_Auditor", "analyzing decision patterns …")
    audit_prompt = (
        f"Planner decision record:\n{json.dumps(decision_record, indent=2)}\n\n"
        f"COAs:\n{json.dumps(coas, indent=2)}\n\n"
        f"Perturbation details:\n"
        + json.dumps({k: v["label"] for k, v in worlds.items()}, indent=2)
    )
    audit_resp = await bias_auditor.on_messages(
        [TextMessage(content=audit_prompt, source="user")],
        cancellation_token=CancellationToken(),
    )
    biases = parse_json_response(audit_resp.chat_message.content)

    if not biases:
        log_success("No cognitive biases detected — decisions are consistent across all worlds.")
        return biases, []

    # ── Explainability Agent ──────────────────────────────────
    log_step("Explainability_Agent", f"generating {len(biases)} appeal brief(s) …")
    explain_prompt = (
        f"Bias diagnoses:\n{json.dumps(biases, indent=2)}\n\n"
        f"COAs:\n{json.dumps(coas, indent=2)}\n\n"
        f"Decision record:\n{json.dumps(decision_record, indent=2)}"
    )
    explain_resp = await explainability_agent.on_messages(
        [TextMessage(content=explain_prompt, source="user")],
        cancellation_token=CancellationToken(),
    )
    briefs = parse_json_response(explain_resp.chat_message.content)

    return biases, briefs


# ── Run Phase 5 ──────────────────────────────────────────────
log_section("Bias Detection & Explainability", phase=5,
            description="The Reveal — surfacing cognitive biases in the planner's reasoning.")
detected_biases, appeal_briefs = await run_bias_analysis(
    baseline_decisions, perturbed_decisions, baseline_coas, counterfactual_worlds
)

# ── Display bias findings ────────────────────────────────────
if detected_biases:
    display(HTML("<h3>⚠️ Cognitive Biases Detected</h3>"))
    for bias in detected_biases:
        color = CONFIDENCE_COLORS.get(bias.get("confidence", ""), "#6e7781")
        display(HTML(
            f"<div style='border:2px solid {color}; border-radius:8px; padding:12px; margin:8px 0;'>"
            f"<h4 style='color:{color}; margin:0;'>⚠ {bias['diagnosed_bias']}</h4>"
            f"<p><b>COA:</b> {bias['coa_id']} &nbsp;|&nbsp; "
            f"<b>Baseline:</b> {bias['baseline_decision']} → "
            f"<b>{bias['flipped_in_world']}:</b> {bias['perturbed_decision']}</p>"
            f"<p><b>Variable changed:</b> {bias['variable_that_changed']}</p>"
            f"<p><b>Reasoning:</b> {bias['reasoning']}</p>"
            f"<p><b>Confidence:</b> <span style='color:{color};font-weight:700;'>"
            f"{bias['confidence'].upper()}</span></p>"
            f"</div>"
        ))
else:
    display(HTML(
        "<div style='padding:12px; background:#dcffe4; border-radius:8px;'>"
        "<b>✓ No biases detected</b> — planner decisions are consistent across all worlds."
        "</div>"
    ))

# ── Display appeal briefs ────────────────────────────────────
if appeal_briefs:
    display(HTML("<h3>📋 Appeal Briefs</h3>"))
    for brief in appeal_briefs:
        held = brief.get("what_stayed_constant", [])
        held_str = ", ".join(held) if isinstance(held, list) else str(held)
        display(HTML(
            f"<div style='border:1px solid #1f6feb; border-radius:8px; padding:12px; "
            f"margin:8px 0; background:#f0f6ff;'>"
            f"<h4 style='color:#1f6feb; margin:0;'>{brief.get('brief_title', 'Appeal')}</h4>"
            f"<p><b>COA:</b> {brief['coa_id']}</p>"
            f"<p><b>Assumption changed:</b> {brief['assumption_changed']}</p>"
            f"<p><b>Held constant:</b> {held_str}</p>"
            f"<p><b>Outcome shift:</b> {brief.get('outcome_shift', 'N/A')}</p>"
            f"<p><b>Original preference valid?</b> {brief.get('original_preference_valid', '?')}</p>"
            f"<p><b>Recommendation:</b> "
            f"<span style='font-weight:700;'>{brief['recommendation'].upper()}</span></p>"
            f"<hr style='border:0;border-top:1px solid #c8d1d9;'/>"
            f"<p>{brief.get('narrative', '')}</p>"
            f"</div>"
        ))

log_success(
    f"Phase 5 complete — {len(detected_biases)} bias(es) detected, "
    f"{len(appeal_briefs)} appeal brief(s) generated."
)

LogEntry(level='SUCCESS', message='Phase 5 complete — 0 bias(es) detected, 0 appeal brief(s) generated.', timestamp='2026-02-12 19:15:47', extra={})

## Full Results — Cross-World Decision Matrix

A single table showing every COA's score and the planner's decision in the baseline and each perturbed world, with flagged biases highlighted.

> **Scaling beyond a notebook:** For production scenarios with many more COAs, worlds, or concurrent users, you could persist these results to [Azure Cosmos DB](https://learn.microsoft.com/azure/cosmos-db/introduction) for low-latency querying or use [Azure Monitor](https://learn.microsoft.com/azure/azure-monitor/overview) to log and visualize decision patterns over time.

In [12]:
log_section("Cross-World Decision Matrix", phase=6,
            description="Every COA's score and decision in baseline + perturbed worlds.")

coa_ids = [c["id"] for c in baseline_coas]

# ── Build HTML table ──────────────────────────────────────────
headers = ["COA", "Name", "Baseline Score", "Baseline Dec"]
for axis, world in counterfactual_worlds.items():
    short = world["label"][:28]
    headers += [f"{short} Score", f"{short} Dec"]
headers.append("Biases Flagged")

table = (
    "<table style='border-collapse:collapse; font-size:12px; width:100%;'>"
    "<tr style='background:#1f6feb; color:#fff;'>"
)
for h in headers:
    table += f"<th style='padding:6px 8px; text-align:left;'>{h}</th>"
table += "</tr>"

for coa in baseline_coas:
    cid = coa["id"]
    row_biases = [b["diagnosed_bias"] for b in detected_biases if b["coa_id"] == cid]
    bg = "#fff3f3" if row_biases else "#ffffff"

    table += f"<tr style='background:{bg};'>"
    table += f"<td style='padding:4px 8px;border:1px solid #ddd;font-weight:700;'>{cid}</td>"
    table += f"<td style='padding:4px 8px;border:1px solid #ddd;'>{coa['name']}</td>"
    table += f"<td style='padding:4px 8px;border:1px solid #ddd;text-align:center;'>{coa['weighted_score']}</td>"

    b_dec = baseline_decisions.get(cid, "?")
    b_icon = "✅" if b_dec == "accept" else "❌"
    table += f"<td style='padding:4px 8px;border:1px solid #ddd;'>{b_icon} {b_dec}</td>"

    for axis, world in counterfactual_worlds.items():
        sc = world["scores"].get(cid, "?")
        p_dec = perturbed_decisions.get(axis, {}).get(cid, "?")
        p_icon = "✅" if p_dec == "accept" else "❌"
        # Highlight flipped decisions
        flipped = (b_dec != p_dec)
        cell_bg = "background:#fff3cd;" if flipped else ""
        table += f"<td style='padding:4px 8px;border:1px solid #ddd;text-align:center;{cell_bg}'>{sc}</td>"
        table += f"<td style='padding:4px 8px;border:1px solid #ddd;{cell_bg}'>{p_icon} {p_dec}</td>"

    bias_str = ", ".join(row_biases) if row_biases else "—"
    table += f"<td style='padding:4px 8px;border:1px solid #ddd;color:#d73a49;'>{bias_str}</td>"
    table += "</tr>"

table += "</table>"
display(HTML(table))

# ── Summary statistics ────────────────────────────────────────
total_decisions = len(coa_ids) * (1 + len(counterfactual_worlds))
flips = len(detected_biases)
unique_biases = set(b["diagnosed_bias"] for b in detected_biases) if detected_biases else set()

log_metric("Total decision points", total_decisions)
log_metric("Inconsistencies detected", flips)
log_metric("Unique biases surfaced", unique_biases if unique_biases else "none")

COA,Name,Baseline Score,Baseline Dec,Weather degrades → dense fog Score,Weather degrades → dense fog Dec,Tank company at 50 % strengt Score,Tank company at 50 % strengt Dec,Enemy reserves arrive in 2 h Score,Enemy reserves arrive in 2 h Dec,ROE tightened — no fires wit Score,ROE tightened — no fires wit Dec,Biases Flagged
COA_A,Rapid Overwhelming Assault,0.775,✅ accept,0.62,✅ accept,0.58,✅ accept,0.557,✅ accept,0.6,✅ accept,—
COA_B,Coordinated Multi-Axis Advance,0.793,✅ accept,0.73,✅ accept,0.68,✅ accept,0.68,✅ accept,0.73,✅ accept,—
COA_C,Deceptive Flanking Maneuver,0.79,✅ accept,0.775,✅ accept,0.74,✅ accept,0.74,✅ accept,0.74,✅ accept,—


LogEntry(level='METRIC', message='Unique biases surfaced: none', timestamp='2026-02-12 19:15:47', extra={'label': 'Unique biases surfaced', 'value': 'none', 'unit': None})

## AAR / Notes

- **What worked well?** — Did the bias detection feel surprising and personal?
- **What surfaced?** — Which biases were identified? Did the audience agree with the diagnosis?
- **Explainability** — Were the appeal briefs clear enough for a reviewer to trace how and why the system reached its conclusions?
- **Key takeaway:** The AI didn't replace the planner — it made the planner's *reasoning visible* for reflection and learning.
- **Scope reminder:** All scenarios, agents, and data are artificially generated for research and educational purposes only. This prototype does not constitute an operational model, validated planning tool, or source of decision authority.

### Further Azure Learning Resources

| Topic | Link |
|---|---|
| Get started with Azure AI Foundry | [Quickstart: Create a project and use the chat playground](https://learn.microsoft.com/azure/ai-studio/quickstarts/get-started-playground) |
| Deploy an OpenAI model | [Deploy Azure OpenAI models](https://learn.microsoft.com/azure/ai-studio/how-to/deploy-models-openai) |
| Manage keys securely with Key Vault | [Azure Key Vault quickstart](https://learn.microsoft.com/azure/key-vault/secrets/quick-create-portal) |
| Run notebooks in Azure ML | [Azure Machine Learning notebooks quickstart](https://learn.microsoft.com/azure/machine-learning/quickstart-run-notebooks) |
| AutoGen multi-agent patterns | [AutoGen AgentChat tutorial](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/quickstart.html) |
| Azure AI Responsible AI practices | [Responsible AI overview](https://learn.microsoft.com/azure/ai-services/responsible-use-of-ai-overview) |
| Understanding token-based pricing | [Azure OpenAI pricing](https://azure.microsoft.com/pricing/details/cognitive-services/openai-service/) |

## Reset / Cleanup

In [ ]:
# Reset all demo state
baseline_coas = []
baseline_decisions = {}
counterfactual_worlds = {}
perturbed_decisions = {}
detected_biases = []
appeal_briefs = []
clear_logs()
log_success("All state cleared — ready for a fresh run.")